In [ ]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import os

os.chdir("../")
os.getcwd()

In [ ]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv

# Load environment variables
load_dotenv(find_dotenv())

# Import required modules
from langchain_openai import AzureChatOpenAI

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0
)


In [ ]:

# # Create a prompt template for math questions.
# math_prompt = ChatPromptTemplate.from_template(
#     """
# You are a helpful math tutor. Please solve the following math problem step by step:

# Question: {math_question}

# Please show your work and explain each step clearly.
# """
# )

# # Create the chain.
# math_chain = LLMChain(llm=llm, prompt=math_prompt, verbose=True)

# # Example usage.
# question = "What is the area of a circle with radius 5 units?"
# response = math_chain.invoke({"math_question": question})
# print(response)

# # Try another example.
# question2 = "Solve the equation: 2x + 5 = 15"
# response2 = math_chain.invoke({"math_question": question2})
# print(response2)

In [4]:
import os
import math
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.tools import StructuredTool
import os
import math
from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

In [5]:


# # Define Pydantic models for input validation
# class CircleAreaInput(BaseModel):
#     """Input schema for circle area calculation."""
#     radius: float = Field(..., description="The radius of the circle.")

# class LinearEquationInput(BaseModel):
#     """Input schema for linear equation solving."""
#     equation: str = Field(..., description="A linear equation of the form 'ax + b = c'.")

# # Update function signatures to match the field names of the Pydantic models.
# def calculate_circle_area(radius: float) -> str:
#     """Calculate the area of a circle given its radius.

#     Args:
#         radius: The radius of the circle in units
#     Returns:
#         The area of the circle in square units
#     """
#     area = math.pi * radius ** 2
#     return f"The area of a circle with radius {radius} is {area:.2f} square units."

# def solve_linear_equation(equation: str) -> str:
#     """
#     Solve a simple linear equation of the form 'ax + b = c'.
#     This function assumes the equation is properly formatted.

#     Args:
#         equation: A linear equation of the form 'ax + b = c'
#     Returns:
#         The solution for x
#     """
#     try:
#         eq = equation.replace(" ", "")
#         x_index = eq.index("x")
#         a = float(eq[:x_index])
#         remainder = eq[x_index + 1:]
#         if remainder[0] in "+-":
#             sign = 1 if remainder[0] == "+" else -1
#             remainder = remainder[1:]
#         else:
#             sign = 1
#         b_str, c_str = remainder.split("=")
#         b = sign * float(b_str)
#         c = float(c_str)
#         x = (c - b) / a
#         return f"The solution for the equation {equation} is x = {x}."
#     except Exception as e:
#         return f"Error parsing the equation '{equation}': {str(e)}"

# # Wrap the functions as StructuredTools using the Pydantic input schemas.
# circle_area_tool = StructuredTool.from_function(
#     func=calculate_circle_area,
#     input_schema=CircleAreaInput,
#     name="CalculateCircleArea",
#     description="Calculate the area of a circle given its radius. Input: a JSON with a 'radius' key."
# )

# linear_equation_tool = StructuredTool.from_function(
#     func=solve_linear_equation,
#     input_schema=LinearEquationInput,
#     name="SolveLinearEquation",
#     description="Solve a linear equation of the form 'ax + b = c'. Input: a JSON with an 'equation' key."
# )

# tools = [circle_area_tool, linear_equation_tool]

In [6]:
# # Bind tools to the model
# llm_with_tools = llm.bind_tools(tools)

# # Create new prompt template for tool calling
# prompt = """You are a helpful AI assistant that uses available tools/functions to solve math problems.
# When given a math problem, analyze it carefully and use the appropriate function to solve it.
# For circle area calculations, use the calculate_circle_area tool.
# For linear equations, use the solve_linear_equation tool.
# Always use the tools instead of doing calculations yourself.

# Question: {input}
# """

# # Test with the same queries
# query1 = "What is the area of a circle with radius 5 units?"
# result1 = llm_with_tools.invoke(prompt.format(input=query1))
# print("\nResult 1:", result1)

# query2 = "Solve the equation: 2x + 5 = 15"
# result2 = llm_with_tools.invoke(prompt.format(input=query2))
# print("\nResult 2:", result2)

In [7]:
def execute_tool_calls(tools_dict: dict, model_output) -> str:
    """
    Execute tool calls based on the model's output.

    Args:
        tools_dict: Dictionary mapping tool names to tool objects
        model_output: Output from the LLM model containing tool calls

    Returns:
        str: Result of executing the tool calls
    """
    if not hasattr(model_output, "tool_calls") or not model_output.tool_calls:
        return "No tool calls found in model output"

    results = []
    for tool_call in model_output.tool_calls:
        tool_name = tool_call["name"]
        if tool_name not in tools_dict:
            results.append(f"Tool {tool_name} not found in available tools")
            continue

        tool = tools_dict[tool_name]
        try:
            result = tool.invoke(tool_call["args"])
            results.append(result)
        except Exception as e:
            results.append(f"Error executing {tool_name}: {str(e)}")

    return "\n".join(results)

In [8]:


# # Create a dictionary mapping tool names to tool objects
# tools_dict = {
#     "CalculateCircleArea": circle_area_tool,
#     "SolveLinearEquation": linear_equation_tool,
# }

# # Test with the previous results
# print("\nExecuting tool calls for Result 1:")
# print(execute_tool_calls(tools_dict, result1))

# print("\nExecuting tool calls for Result 2:")
# print(execute_tool_calls(tools_dict, result2))

#### now we try out with new tools imported from utils

In [9]:
from utils.web_utils import (
    get_stock_symbol_alpha_vantage,
    get_company_financials,
    get_news_headlines,
)

In [10]:
from typing import Optional
from langchain_core.callbacks import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field

class CompanyFinancialsInput(BaseModel):
    company_name: str = Field(description="name of the company to get financials for")

class CompanyNewsInput(BaseModel):
    company_name: str = Field(description="name of the company to get news for")

class CompanyFinancialsTool(BaseTool):
    name: str = "GetCompanyFinancials"
    description: str = "Get financial information for a company by name"
    args_schema: Optional[type[BaseModel]] = CompanyFinancialsInput
    
    def _run(
        self, 
        company_name: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        if not company_name:
            return "Company name is required"
            
        # First get the stock symbol
        symbols = get_stock_symbol_alpha_vantage(company_name)
        if not symbols:
            return f"No stock symbol found for {company_name}"
            
        # Use first symbol to get financials
        symbol = symbols[0]["symbol"]
        financials = get_company_financials(symbol)
        return financials

    async def _arun(
        self,
        company_name: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        return self._run(company_name, run_manager=run_manager.get_sync())

class CompanyNewsTool(BaseTool):
    name: str = "GetCompanyNews"
    description: str = "Get recent news headlines for a company by name"
    args_schema: Optional[type[BaseModel]] = CompanyNewsInput

    def _run(
        self,
        company_name: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        if not company_name:
            return "Company name is required"
            
        headlines = get_news_headlines(company_name)
        return headlines

    async def _arun(
        self,
        company_name: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        return self._run(company_name, run_manager=run_manager.get_sync())


In [22]:
# Create instances of tools
company_financials_tool = CompanyFinancialsTool()
company_news_tool = CompanyNewsTool()

tools_dict = {
    "GetCompanyFinancials": company_financials_tool,
    "GetCompanyNews": company_news_tool,
}

# Create tools list
tools = [company_financials_tool, company_news_tool]

# Initialize chat model with tools
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [23]:

# Create messages list for conversation
from langchain_core.messages import HumanMessage, SystemMessage


def get_prompt(question):

    # Define the system message that explains available tools and agent's role
    system_message = """You are a financial research assistant with access to two specialized tools:

    1. GetCompanyFinancials:
    - Purpose: Retrieves detailed financial information for a company
    - Input: Company name (e.g., "Apple", "Microsoft")
    - Use this when users ask about financial data, earnings, or financial metrics

    2. GetCompanyNews:
    - Purpose: Fetches recent news headlines about a company
    - Input: Company name (e.g., "Apple", "Microsoft")
    - Use this when users ask about recent news, updates, or company developments

    Your task is to:
    1. Analyze the user's question carefully
    2. Determine which tool(s) would be most appropriate
    3. Use the tool(s) to gather relevant information
    4. Provide a clear, concise response based on the information retrieved

    Always use the appropriate tools rather than making assumptions or providing general information.
    """

    # Create the messages for the conversation
    messages = [SystemMessage(content=system_message), HumanMessage(content=question)]

    return messages

In [27]:
question = "What is the financial information for Apple?"
question = "What is the latest news for Microsoft?"
messages = get_prompt(question)
ai_msg = llm_with_tools.invoke(messages)

In [ ]:
ai_msg

In [ ]:
output = execute_tool_calls(tools_dict, ai_msg)
output